In [13]:
from datetime import datetime
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [14]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [15]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [16]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
col_conn = create_engine(url_db)

## Load table sede

In [17]:
sede = pd.read_sql_table("sede",url_db)
ciudad = pd.read_sql_table("ciudad",url_db)
departamento = pd.read_sql_table("departamento",url_db)

In [18]:
ciudad.rename(columns={"nombre": "nombre_cuidad"}, inplace=True)
ciudad.head()

,ciudad_id,nombre_cuidad,departamento_id
0,6,BUGA,1
1,5,BOGOTA,2
2,4,PASTO,4
3,3,POPAYAN,3
4,2,PALMIRA,1


In [19]:
sede = sede.merge(
    ciudad,
    left_on='ciudad_id',  # Columna en el DataFrame de mensajeros
    right_on='ciudad_id',                   # Columna en el DataFrame de ciudades
    how='left'                       # Mantiene todos los registros del DataFrame de mensajeros
)
sede.head()

,sede_id,nombre,direccion,telefono,nombre_contacto,ciudad_id,cliente_id,nombre_cuidad,departamento_id
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,3,11,POPAYAN,3


In [20]:
departamento.rename(columns={"nombre": "nombre_departamento"}, inplace=True)
departamento

,departamento_id,nombre_departamento
0,4,NARIÑO
1,3,CAUCA
2,2,CUNDINAMARCA
3,1,VALLE DEL CAUCA


In [21]:
sede = sede.merge(
    departamento,
    left_on='departamento_id',  # Columna en el DataFrame de mensajeros
    right_on='departamento_id',                   # Columna en el DataFrame de ciudades
    how='left'                       # Mantiene todos los registros del DataFrame de mensajeros
)
sede.head()

,sede_id,nombre,direccion,telefono,nombre_contacto,ciudad_id,cliente_id,nombre_cuidad,departamento_id,nombre_departamento
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1,VALLE DEL CAUCA
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1,VALLE DEL CAUCA
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1,VALLE DEL CAUCA
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4,CALI,1,VALLE DEL CAUCA
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,3,11,POPAYAN,3,CAUCA


In [22]:
sede.drop(columns=["ciudad_id","departamento_id"], inplace=True)
sede["saved"] = datetime.now()
sede.head()

,sede_id,nombre,direccion,telefono,nombre_contacto,cliente_id,nombre_cuidad,nombre_departamento,saved
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,CALI,VALLE DEL CAUCA,2024-10-30 08:14:09.110207
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,CALI,VALLE DEL CAUCA,2024-10-30 08:14:09.110207
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,CALI,VALLE DEL CAUCA,2024-10-30 08:14:09.110207
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,4,CALI,VALLE DEL CAUCA,2024-10-30 08:14:09.110207
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,11,POPAYAN,CAUCA,2024-10-30 08:14:09.110207


## Load table into ETL database

In [23]:
sede.to_sql("dim_sede", etl_conn, index_label="key_sede", if_exists="replace") 

52